# Summary 18

# Домашнее задание 35
1. **Предсказание пола по имени**  
Изучите документацию API предсказания пола по имени: genderize.io/documentation. Напишите программу, которая запрашивает имя пользователя и выводит его пол и вероятность этого предсказания.

*Пример ввода:*   
Введите имя: Sasha  
Пример вывода:   
Предполагаемый пол: female  
Вероятность: 59.0%


In [5]:
import requests

name = input("Введите имя: ")

url = "https://api.genderize.io"
params = {"name": name}

response = requests.get(url, params=params)

data = response.json()


if response.status_code == 200:
    data = response.json()
    gender = data["gender"]
    probability = data["probability"]

    if gender:
        print(f"Предполагаемый пол: {gender}")
        print(f"Вероятность: {float(probability) * 100:.1f}%")
    else:
        print("Пол не удалось определить.")
else:
    print("Ошибка при обращении к API:", response.status_code)


Введите имя:  Sasha


Предполагаемый пол: female
Вероятность: 59.0%


2. **Получение биржевых данных через API**   
Напишите программу, которая запрашивает у пользователя тикер (symbol) акций, обращается к функции с помесячными данными о результатах торгов (TIME_SERIES_MONTHLY) в API ресурса www.alphavantage.co, запрашивает результат в формате csv, записывает его в файл. Доступность ресурса предварительно нужно проверить. После этого файл требуется прочитать, извлечь данные из столбца volume и вычислить среднее значение этого столбца.

 *Пример ввода:*   
Введите тикер: BMWYY

*Пример вывода:*   
CSV-файл успешно сохранён  
Средний объём: 7638446.53


In [10]:
import requests, csv
import statistics as st

symbol = input("Введите тикер: ")

params = {
    'function' : 'TIME_SERIES_MONTHLY',
    'symbol' : symbol,
    'apikey' : 'F3Y69VXR8G9EYNI8',
    'datatype' : 'csv'
    }

url = 'https://www.alphavantage.co/query'
response = requests.get(url, params)

if response.status_code == 200:
    resp_text = response.text
else:
    print("Ошибка запроса:", response.status_code)
    exit()


with open("output_"+symbol+".csv", "w") as f:
    f.write(resp_text)  
print("CSV-файл успешно сохранён")

volumes = []
with open("output_"+symbol+".csv", "r") as f:
    reader = csv.DictReader(f)
    for row in reader:
        value = row.get("volume")
        volumes.append(float(value))
print(f"Средний объём: {st.mean(volumes):.2f}")


Введите тикер:  BMWYY


CSV-файл успешно сохранён
Средний объём: 7638446.53


# Домашнее задание 36
1. **Поиск стран**  
Напишите программу, которая:  
Парсит таблицу с сайта https://www.iban.com/country-codes и сохраняет ее в подходящей структуре данных;  
Запрашивает у пользователя код страны (пользователь может ввести двухбуквенный или трехбуквенный код) и выводит название страны, соответствующей этому коду.

*Пример ввода:*  
Введите код страны (2 или 3 буквы):  FRA  
*Пример вывода:*  
Код FRA соответствует стране: France

In [ ]:
import requests, statistics
from bs4 import BeautifulSoup

url = "https://contextures.com/xlsampledata01.html"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    
    # На странице много таблиц. Нас интересует третья
    tables = soup.find_all("table")
    table = tables[2]

    rows = table.find_all("tr")
    
    # Получаем заголовки
    headers = [td.get_text(strip=True) for td in rows[0].find_all("td")]
    
    # Получаем строки таблицы и записываем их значения в разные списки
    columns = {key: [] for key in headers}
    for row in rows[1:]:
        cells = row.find_all("td")
        for header, cell in zip(headers, cells):
            if header in ["Units", "UnitCost", "Total"]:
                columns[header].append(float(cell.text.strip()))
            else:
                columns[header].append(cell.text.strip())

    # Вывод результатов
    print(f"Средняя цена товара: {statistics.mean(columns["UnitCost"])}")
    print(f"Суммарная выручка: {sum(columns["Total"])}")

else:
    print("Не удалось загрузить страницу:", response.status_code)

